# Trying everything again
using xpath

### Get a categorized list of all the pages

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import os
from lxml import html
import re
from urllib.parse import unquote
import time
from http.client import RemoteDisconnected
from urllib3.exceptions import MaxRetryError
from requests.exceptions import Timeout

d_process = 0
retries = 3
delay = 5
base_url = "https://eci.gov.in"
# use this
def get_json():
    lists = {} # dictionary to store all the links
    for i in range(0,4):
        time.sleep(delay)
        linkss = {}
        if(i==0):
            
            response = requests.get("https://eci.gov.in/statistical-report/statistical-reports/")
            soup = BeautifulSoup(response.content, 'html.parser')
            body = soup.find(id="elCmsPageWrap").select ('table')[i].select('em')[0].text.strip()
            # print (body)
            for link in soup.find(id="elCmsPageWrap").select ('table')[i].select('a'):
                name = link.text.strip()
                link = link.get('href')
                linkss[name] = link
        if(i==1):
            response = requests.get("https://eci.gov.in/statistical-report/statistical-reports/")
            soup = BeautifulSoup(response.content, 'html.parser')
            body = soup.find(id="elCmsPageWrap").select ('table')[1].select('em')[0].text.strip()
            states = {}
            state_soup = soup.find(id="elCmsPageWrap").select ('table')[1].select('tr')
            state_soup = state_soup[1:]
            for one in state_soup:
                two = one.select('td')[0]
                one = one.select('td')[1]
                link_a = one.select('a') 
                link_name = {}
                # print (two.text.strip())
                for linka in link_a:
                    name = linka.text.strip()
                    link = linka.get('href')
                    # print ('\t',name, link)
                    link_name[name] = link
                states[two.text.strip()] = link_name
            linkss = states
        #     print (linkss)
        # else: continue ## to be removed
        if(i==2):
            response = requests.get("https://eci.gov.in/statistical-report/statistical-reports/")
            soup = BeautifulSoup(response.content, 'html.parser')
            body = soup.find(id="elCmsPageWrap").select ('table')[i].select('em')[0].text.strip()
            soup = soup.find(id="elCmsPageWrap").select ('table')[i].select('tr')
            states = {}
            for i in range(1,len(soup)):
                links = {}
                state = soup[i].select('td')[0].text.strip()
                for link in soup[i].select('td')[1].select('a'):
                    name = link.text.strip()
                    link = link.get('href')
                    links[name] = link
                # print (state, links)
                states[state] = links
                state = soup[i].select('td')[2].text.strip()
                for link in soup[i].select('td')[3].select('a'):
                    name = link.text.strip()
                    link = link.get('href')
                    links[name] = link
                # print (state, links)
                states[state] = links
            linkss = states
        if(i==3):
            response = requests.get("https://eci.gov.in/statistical-report/statistical-reports/")
            soup = BeautifulSoup(response.content, 'html.parser')
            body = soup.find(id="elCmsPageWrap").select ('table')[i].select('em')[0].text.strip()
            soup = soup.find(id="elCmsPageWrap").select ('table')[3].select('tr')
            category = {}
            new_soup = soup
            links = {}
            names = []
            link = []
            for k in range (1,len(new_soup)-1):
                name = new_soup[k].select('td')[0].select('a')
                for j in name:
                    names.append(j.text.strip())
                    link.append(j.get('href'))
            for k in range (0,len(names)):
                links[names[k]] = link[k]
            category['GENERAL ELECTION TO STATE LEGISLATIVE ASSEMBLY'] = links
            links = {}
            names = []
            link = []
            for k in range (1,len(new_soup)-1):
                name = new_soup[k].select('td')[1].select('a')
                for j in name:
                    names.append(j.text.strip())
                    link.append(j.get('href'))
            for k in range (0,len(names)):
                links[names[k]] = link[k]
            category['GENERAL ELECTION TO LOK SABHA'] = links
            linkss = category
        lists[body] = linkss
    # write to json file
    with open('links.json', 'w') as f:
        json.dump(lists, f, indent=4)
    return lists


### Downloading the file

In [50]:

def fetch_url(url):
    if base_url not in url:
        url = base_url+url
    for i in range(retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response
        except (requests.exceptions.RequestException, ConnectionError, OSError, ConnectionAbortedError, Timeout, RemoteDisconnected, MaxRetryError)as e:
            # print("Connection er: ",e)
            # time.sleep(delay)
            print ("Retrying...")
    print ("Failed after ",retries," retries")
    return None

def req(xpath,u): # pass a single link
    if u == None:
        return None
    if base_url not in u:
        u = base_url+u
    
    response = requests.get(u) ###
    response = fetch_url(u)
    if response!=None:
        tree = html.fromstring(response.content)
        components = tree.xpath(xpath)
        if components:
            return components[0].get('href')
    else:
        return None

def link_to_down(link): # pass a single link
    if type (link) == list and len(link) == 1:
        return link
    elif type (link) == list and len(link) > 1:
        link = link[0]
    if link == None:
        return None
    if base_url not in link:
        link = base_url+link
    link_list = []
    xpath = '/html/body/main/div[3]/div/div/div/div[2]/div[1]/aside/div/ul[1]/li/a' # download this file
    links_req = link
    link_req = req(xpath,links_req) # download this file
    if link_req!=None:
        links_req = link_req
    xpath = '/html/body/main/div[3]/div/div/div/div[2]/ul/li[2]/a' # agree & download
    link_req = req(xpath,links_req) # agree & download
    i=1
    if link_req!=None:
        links_req = link_req
    xpath = f'/html/body/main/div[3]/div/div/div/div/ul/li[{i}]/div[2]/a'
    link_req = req(xpath,links_req) # download
    if link_req==None:
        return None
    while True:
        xpath = f'/html/body/main/div[3]/div/div/div/div/ul/li[{i}]/div[2]/a' # download
        link_req = req(xpath,links_req)
        if link_req==None:
            break
        link_list.append(link_req)
        i+=1
    return link_list

def fix(input_string):
    # Replace symbols (excluding . and /) with underscores
    fixed_string = re.sub(r'[^a-zA-Z0-9]', '', input_string)
    # Replace double backslashes with a single forward slash
    fixed_string = fixed_string.replace('\\', '/')
    # Alternatively, you can replace double backslashes with a double backslash
    # fixed_string = fixed_string.replace('\\\\', '\\\\')
    
    return fixed_string

def fix_file_name(input_string):
    # Replace symbols (excluding . and /) with underscores
    fixed_string = re.sub(r'[^a-zA-Z0-9.]', '', input_string)
    # Replace double backslashes with a single forward slash
    fixed_string = fixed_string.replace('\\', '/')
    # Alternatively, you can replace double backslashes with a double backslash
    # fixed_string = fixed_string.replace('\\\\', '\\\\')
    
    return fixed_string

# use this
def get_file_name_from_url(url):
    
    response = requests.head(url)
    content_disposition = response.headers.get("content-disposition")
    if content_disposition and "filename=" in content_disposition:
        file_name = unquote(content_disposition.split("filename=")[1])
    else:
        file_name = url.split("/")[-2]
        file_name = file_name + ".pdf"
    file_name = fix_file_name(file_name)
    return file_name

def download_file(urls,directory):
    
    urls = link_to_down(urls)
    if urls == None:
        print ("No file", urls)
        return
    for url in urls:
        if url == None:
            print ("No file", urls)
            continue
        filename = get_file_name_from_url(url)
        file_path = os.path.join(directory, filename)
        file_path = file_path.replace('\\', '/')
        # file_path = os.path.join(file_path, "file.txt")
        i=0
        while(i<retries):
            i+=1
            try:
                # check if file_path exists
                if not os.path.exists(os.path.dirname(file_path)):
                    # build file_path or file path
                    response = requests.get(url)
                    os.makedirs(os.path.dirname(file_path))
                else: break
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                break
            except (requests.exceptions.RequestException, ConnectionError, OSError, ConnectionAbortedError, Timeout, RemoteDisconnected, MaxRetryError)as e:
                # print("Connection er: ",e)
                # time.sleep(delay)
                print ("Retrying Download...")


### Expanding within categories

In [51]:

# new
def broaden_save(link, directory):
    if type(link)==type([]):
        print ("Error: pass a single link")
        print (link)
        return
    links = link
    if links == None:
        return
    if base_url not in links:
        links = base_url+links
    list_links = []
    skip = []
    if type(links)==type([]):
        if len(links)==1:
            links =links[0]
    temp_link=link_to_down(links)
    if temp_link!=None or temp_link!=[]:
        download_file(temp_link, directory)
    else: # navigate through the paginated list
        # first page
        for j in range(1,26):
            list_link = req(f'/html/body/main/div[3]/div/div[1]/div[2]/div/div/ol/li[{j}]/div[1]/a',links)
            if list_link==None:
                skip.append(j)
                continue
            temp_link = list_link
            if temp_link!=None:
                list_links.append(temp_link)
        # other pages
        cur_page = links
        next_page = req('/html/body/main/div[3]/div/div[1]/div[2]/div/div/div[1]/div/ul/li[5]/a',cur_page)
        while (next_page!=cur_page):
            cur_page = next_page
            next_page = req('/html/body/main/div[3]/div/div[1]/div[2]/div/div/div[1]/div/ul/li[5]/a',cur_page)
            # next pages
            for j in range(1,26):
                list_link = req(f'/html/body/main/div[3]/div/div[1]/div[2]/div/div/ol/li[{j}]/div[2]/h4/span/a',cur_page)
                if list_link==None:
                    skip.append(j)
                    continue
                temp_link = list_link
                if temp_link!=None:
                    list_links.append(temp_link)
        # all list link added to list_links
    if len(skip)==50:
        download_file(links, directory)
    for link in list_links:
        download_file(link, directory)

def expand_save(directory, dict):
    if not os.path.exists(directory):
        os.makedirs(directory)
    for key in dict.keys():
        if type(dict[key])==type({}):
            expand_save(directory+'/'+fix(key),dict[key])
        elif type(dict[key])==type([]):
            for link in dict[key]:
                broaden_save(link, directory+'/'+fix(key))
        elif dict[key]==None:
            continue
        else:
            broaden_save(dict[key], directory+'/'+fix(key))


### Getting everything together

In [4]:
lists = get_json()
lists["Miscellaneous"] = "https://eci.gov.in/files/category/968-miscellaneous-statics/"
lists["General Election 2014"] = "https://eci.gov.in/files/category/97-general-election-2014/"
lists["General Election 2019 (Excluding Vellore PC)"] = "https://eci.gov.in/files/category/1359-general-election-2019-excluding-vellore-pc/"


### Almost done

In [22]:
# save lists to json file
with open('links.json', 'w') as f:
    json.dump(lists, f, indent=4)
    

In [67]:
# create dictionary from json file
# file is links.json
with open('links.json') as f:
    lists = json.load(f)


### Scrapping has not been done for the following pages:
- https://eci.gov.in/statistical-report/link-to-form-20/
- https://eci.gov.in/statistical-report/detailed-bye-election-results/
- https://eci.gov.in/bye-election/bye-election-result/


In [68]:

lists = expand_save('./ECI_data',lists)


No file None


# The END